<a id='task4'></a>
<div class=" alert alert-warning">
    <b>Assigment.</b> <b>Task 4.</b>

Use features calculated for each modality in task 3. Choose base classifier for each modality from scikit-learn. Train classifiers for each modality feature presentations separately and combine the outputs in decision level. Evaluate and compare the result on testing dataset. Do the subtasks given as
<br>
<br>
<p> <b>4.1</b> Use base classifiers of support vector machine (SVM) with RBF-kernel and AdaBoost classifier (with random_state=0).
Normalize data between 0-1 using the training dataset. Train the base classifiers by tuning the model parameters (<i>C</i> parameter and RBF-kernel <i>gamma</i> in SVM as well as <i>n_estimators</i> and <i>learning_rate</i> in Adaboost) using 10-fold cross-validation on training dataset to find optimal set of parameters (hint: use GridSearchCV from scikit-learn). For grid search use the following values $C = [0.1, 1.0, 10.0, 100.0]$, $gamma=[0.1, 0.25, 0.5, 0.75, 1.0, 2.0]$, $n\_estimators = [50, 100, 500, 1000]$, and $learning\_rate = [0.1, 0.25, 0.5, 0.75,1.0]$. Choose the best parameters and train the classifiers for each modality on whole training dataset. Is there a possibility that classifiers will overfit to training data using this parameter selection strategy? If so, why? </p>
<br>
<p> <b>4.2</b> Predict probabilistic outputs of each trained classifier for both modalities using the test set. </p>
<br>
<p> <b>4.3</b> Combine the probabilistic outputs of different modalities by fixed classification rules: max, min, prod, and sum. Evaluate, compare, and analyse the final combined results using confusion matrices and F1 scores. Show results for each base classifier combinations (i.e., $SVM_{acc}+SVM_{depth}$, $AdaBoost_{acc}+AdaBoost_{depth}$, $SVM_{acc}+AdaBoost_{depth}$, $AdaBoost_{acc}+SVM_{depth}$)</p>
<br>
Document your work, evaluate the results, and analyse the outcomes in each subtasks 4.1-4.3.

</div>

In [ ]:
import pandas as pd
import numpy as np
import time
from copy import deepcopy
import importlib, utilities.fun_four, utilities.fun_one
importlib.reload(utilities.fun_four)
importlib.reload(utilities.fun_one)
from utilities.fun_four import DataNormalizer, GridClassifier, svm_classify, ada_classify, combine_probabilities, combine_visualize
from utilities.fun_one import visualize

In [ ]:
#Get the training data form the notebook slovo_one:
%store -r train_records_merged
df_records_windowed = train_records_merged
%store -r test_records_merged
df_records_windowed = test_records_merged

## 4.1

In [ ]:
start_time = time.time()

act_train = np.concatenate(train_records_merged.df_y.values)
act_test = np.concatenate(test_records_merged.df_y.values)
act_train_labels = np.array(train_records_merged.exercise_id.apply(lambda x: int(x)))
act_test_labels = np.array(test_records_merged.exercise_id.apply(lambda x: int(x)))


dc_train = np.concatenate(train_records_merged.df_x.values)
dc_test = np.concatenate(test_records_merged.df_x.values)
dc_train_labels = np.array(train_records_merged.exercise_id.apply(lambda x: int(x)))
dc_test_labels = np.array(test_records_merged.exercise_id.apply(lambda x: int(x)))

end_time = time.time()
print("Execution Time: ", end_time - start_time)

In [ ]:
start_time = time.time()

# Normalize the data:
act_normalizer = DataNormalizer()
act_normalizer.fit(act_train)
act_train = act_normalizer.transform(act_train)
act_test = act_normalizer.transform(act_test)

dc_normalizer = DataNormalizer()
dc_normalizer.fit(dc_train)
dc_train = dc_normalizer.transform(dc_train)
dc_test = dc_normalizer.transform(dc_test)

end_time = time.time()
print("Execution Time: ", end_time - start_time)

In [ ]:
start_time = time.time()

# Classification accelerometer data:
act_svm_best = svm_classify(act_train,act_train_labels,act_test)

end_time = time.time()
print("Execution Time: ", end_time - start_time)

In [ ]:
start_time = time.time()

act_ada_best = ada_classify(act_train,act_train_labels,act_test)

end_time = time.time()
print("Execution Time: ", end_time - start_time)


In [ ]:
start_time = time.time()

# Classification depth camera data:
dc_svm_best  = svm_classify(dc_train,dc_train_labels,dc_test)

end_time = time.time()
print("Execution Time: ", end_time - start_time)

In [ ]:
start_time = time.time()

dc_ada_best = ada_classify(dc_train,dc_train_labels,dc_test)

end_time = time.time()
print("Execution Time: ", end_time - start_time)


## 4.2

In [ ]:
start_time = time.time()

# 4.2 Get the predictions:
act_svm_pred_train = act_svm_best.predict(act_train)
act_svm_pred_test = act_svm_best.predict(act_test)
act_svm_proba_train = act_svm_best.predict_proba(act_train)
act_svm_proba_test = act_svm_best.predict_proba(act_test)

act_ada_pred_train = act_ada_best.predict(act_train)
act_ada_pred_test = act_ada_best.predict(act_test)
act_ada_proba_train = act_ada_best.predict_proba(act_train)
act_ada_proba_test = act_ada_best.predict_proba(act_test)

dc_svm_pred_train =  dc_svm_best.predict(dc_train)
dc_svm_pred_test =   dc_svm_best.predict(dc_test)
dc_svm_proba_train = dc_svm_best.predict_proba(dc_train)
dc_svm_proba_test =  dc_svm_best.predict_proba(dc_test)

dc_ada_pred_train =  dc_ada_best.predict(dc_train)
dc_ada_pred_test =   dc_ada_best.predict(dc_test)
dc_ada_proba_train = dc_ada_best.predict_proba(dc_train)
dc_ada_proba_test =  dc_ada_best.predict_proba(dc_test)

end_time = time.time()
print("Execution Time: ", end_time - start_time)

In [ ]:
# Visualisation Accelerometer with svm:
visualize(act_svm_pred_train,act_train_labels,act_svm_pred_test,act_test_labels,main_title="Accelerometer sensor used with svm to classify the exercises")

In [ ]:
# Visualisation Accelerometer with ada boost:
visualize(act_ada_pred_train,act_train_labels,act_ada_pred_test,act_test_labels,main_title="Accelerometer sensor used with ada boost to classify the exercises")

In [ ]:
# Visualisation Depth Camera with svm:
visualize(dc_svm_pred_train,dc_train_labels,dc_svm_pred_test,dc_test_labels,main_title="Depth Camera sensor used with svm to classify the exercises")

In [ ]:
# Visualisation Depth Camera with ada boost:
visualize(dc_ada_pred_train,dc_train_labels,dc_ada_pred_test,dc_test_labels,main_title="Depth Camera sensor used with ada boost to classify the exercises")

## 4.3


In [ ]:
start_time = time.time()

ground_truth_train = act_train_labels # They are equal to the dc_train_labels
ground_truth_test = act_test_labels

# Combine predictions SVM-act and SVM-dc
svm2_train, svm2_test = combine_probabilities(act_svm_proba_train,dc_svm_proba_train,
                                              act_svm_proba_test,dc_svm_proba_test)
# Combine predictions SVM-act and Ada-dc
svmad_train, svmad_test = combine_probabilities(act_svm_proba_train,dc_ada_proba_train,
                                              act_svm_proba_test,dc_ada_proba_test)
# Combine predictions Ada-act and Ada-dc
ada2_train, ada2_test = combine_probabilities(act_ada_proba_train,dc_ada_proba_train,
                                              act_ada_proba_test,dc_ada_proba_test)
# Combine predictions Ada-act and SVM-dc
adasv_train, adasv_test = combine_probabilities(act_ada_proba_train,dc_ada_proba_train,
                                              act_ada_proba_test,dc_ada_proba_test)

end_time = time.time()
print("Execution Time: ", end_time - start_time)

In [ ]:
#Visualize
# Combine predictions SVM-act and SVM-dc
combine_visualize(svm2_train,ground_truth_train,svm2_test,ground_truth_test,"SVM-act and SVM-dc")

In [ ]:
# Combine predictions SVM-act and Ada-dc
combine_visualize(svmad_train,ground_truth_train,svmad_test,ground_truth_test,"SVM-act and Ada-dc")

In [ ]:
# Combine predictions Ada-act and Ada-dc
combine_visualize(ada2_train,ground_truth_train,ada2_test,ground_truth_test,"Ada-act and Ada-dc")

In [ ]:
# Combine predictions Ada-act and SVM-dc
combine_visualize(adasv_train,ground_truth_train,adasv_test,ground_truth_test,"Ada-act and SVM-dc")